In [36]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Load the sales dataset
sales_file_path = '/content/Data.xlsx'
try:
    sales_df = pd.read_excel(sales_file_path, sheet_name='Final', engine='openpyxl')
except Exception as e:
    print("Error loading sales file. Ensure the file format is valid and engine is installed.")
    raise e

# Reshape sales data from wide to long format
sales_df_long = sales_df.melt(id_vars=["Warehouse id", "Region", "SKU id"],
                              var_name="Date",
                              value_name="Sales")

# Convert the 'Date' column to datetime format
sales_df_long['Date'] = pd.to_datetime(sales_df_long['Date'])

# Sort the data for time series processing
sales_df_long = sales_df_long.sort_values(by=["Warehouse id", "SKU id", "Date"])

# Fill missing values (if any) with 0
sales_df_long['Sales'] = sales_df_long['Sales'].fillna(0)

# Feature Engineering: Add lag and rolling mean features for sales
def create_features(data, lags, rolling_windows):
    for lag in lags:
        data[f'lag_sales_{lag}'] = data.groupby(['Warehouse id', 'SKU id'])['Sales'].shift(lag)
    for window in rolling_windows:
        data[f'rolling_mean_sales_{window}'] = data.groupby(['Warehouse id', 'SKU id'])['Sales'].shift(1).rolling(window).mean()
    return data

# Add lag and rolling mean features
sales_df_long = create_features(sales_df_long, lags=[1, 2, 3], rolling_windows=[2, 3])

# Drop rows with NaN values (caused by lag/rolling features)
sales_df_long = sales_df_long.dropna()

# Split data into training and testing sets
train = sales_df_long[sales_df_long['Date'] < '2021-06-01']
test = sales_df_long[sales_df_long['Date'] == '2021-06-01']

# Define features and target variable
features = [col for col in sales_df_long.columns if col.startswith('lag_') or col.startswith('rolling_mean_')]
target = 'Sales'

# Train a separate model for each SKU-Warehouse combination
predictions = []
for (warehouse, sku), group in train.groupby(['Warehouse id', 'SKU id']):
    # Train-test split
    X_train = group[features]
    y_train = group[target]

    # Test data for this SKU-Warehouse
    test_group = test[(test['Warehouse id'] == warehouse) & (test['SKU id'] == sku)]
    if not test_group.empty:
        X_test = test_group[features]

        # Train the model
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Predict for June 2021
        test_group['Forecasted Sales'] = model.predict(X_test)
        predictions.append(test_group)


    else:
        # Handle cases where there is no test data
        empty_prediction = group.iloc[-1:].copy()
        empty_prediction['Date'] = pd.Timestamp('2021-06-01')
        empty_prediction['Forecasted Sales'] = y_train.mean()  # Default to mean sales
        predictions.append(empty_prediction)

# Combine all predictions
if predictions:
    predictions_df = pd.concat(predictions)
else:
    raise ValueError("No predictions were generated. Please check your data and feature engineering steps.")

# Load the submission format
submission_file_path = '/content/Submission Format.xlsx'  # Replace with the correct file path
submission_df = pd.read_excel(submission_file_path)

# Merge the predictions into the submission file
submission_df = submission_df.merge(predictions_df[['Warehouse id', 'SKU id', 'Forecasted Sales']],
                                     on=['Warehouse id', 'SKU id'],
                                     how='left')

# Rename the forecasted sales column for submission
submission_df.rename(columns={'Forecasted Sales': 'June 2021 Sales'}, inplace=True)

# Fill missing values in case any SKU/Warehouse combination didn't have predictions
submission_df['June 2021 Sales'] = submission_df['June 2021 Sales'].fillna(0)

# Save the submission file
submission_df.to_excel('Abhirup.xlsx', index=False)

# Output the final submission file location
print("Submission file has been created: Abhirup.xlsx")


Submission file has been created: Abhirup.xlsx


In [37]:
import pandas as pd
import numpy as np

# Function to calculate MAPE
def calculate_mape(actual, forecasted):
    actual, forecasted = np.array(actual), np.array(forecasted)
    # Avoid division by zero
    non_zero_indices = actual != 0
    if non_zero_indices.sum() == 0:  # If there are no non-zero actual values
        return np.nan
    mape = np.mean(np.abs((actual[non_zero_indices] - forecasted[non_zero_indices]) / actual[non_zero_indices])) * 100
    return mape

# Load the submission format (actual sales for June 2021)
checking_file_path = '/content/Abhirup.xlsx'
checking_df = pd.read_excel(checking_file_path)

# Ensure the column names match
checking_df.rename(columns={"June 2021 Sales": "Sales"}, inplace=True)

# Merge the predictions into the submission file
# Ensure predictions_df is correctly generated from your model predictions
merged_df = checking_df.merge(
    predictions_df[['Warehouse id', 'SKU id', 'Forecasted Sales']],
    on=['Warehouse id', 'SKU id'],
    how='left'
)

# Check for missing data
if merged_df['Forecasted Sales'].isnull().any():
    print("Warning: Some SKUs are missing forecasted sales. These rows will be excluded from MAPE calculation.")

# Drop rows with missing forecasted sales
merged_df = merged_df.dropna(subset=['Forecasted Sales'])

# Calculate MAPE
actual_sales = merged_df['Sales'].values
forecasted_sales = merged_df['Forecasted Sales'].values

mape_value = calculate_mape(actual_sales, forecasted_sales)

# Output the MAPE value
print(f"Mean Absolute Percentage Error (MAPE): {mape_value:.2f}%")


Mean Absolute Percentage Error (MAPE): 0.00%
